In [103]:
# required packages
import os
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from keras.models import Sequential
from keras.layers import Dense
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from keras.layers import PReLU
import numpy as np
import pandas as pd
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import Ridge
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, LSTM, Input, Dropout, BatchNormalization
from tensorflow.keras.models import Sequential, Model
from keras.models import Model, Sequential
from keras.layers import LSTM, Dense, Input, Flatten, Conv1D,Dropout, MaxPooling1D
from keras.applications import VGG16
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.callbacks import EarlyStopping
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder


In [104]:
# Loading data
data_folder = 'data/P_data'
data_files = [f for f in os.listdir(data_folder) if f.endswith('.csv')]

data_dict = {}

for file in data_files:
    name = file.split('.')[0]
    data = pd.read_csv(os.path.join(data_folder,file))
    data_dict[name] = data

bray_global = data_dict['bray_global']
bray_afrlac = data_dict['bray_afrlac']
olsen_global = data_dict['olsen_global']
olsen_afrlac = data_dict['olsen_afrlac']
stp_global = data_dict['stp_global']
stp_afrlac = data_dict['stp_afrlac']

# Instantiate the encoder
encoder = LabelEncoder()

# Check if 'WRB.SOIL.TYPE' exists in the DataFrame, then transform it
if 'WRB.SOIL.TYPE' in stp_global.columns:
    stp_global['WRB.SOIL.TYPE'] = encoder.fit_transform(stp_global['WRB.SOIL.TYPE'])
if 'WRB.SOIL.TYPE' in stp_afrlac.columns:
    stp_afrlac['WRB.SOIL.TYPE'] = encoder.transform(stp_afrlac['WRB.SOIL.TYPE'])  # use transform, not fit_transform

In [105]:
def prepare_data(df):
    # Separate the predictors and response
    X = df.drop(['p_avg', 'LONGITUDE', 'LATITUDE', 'GEO3major'], axis=1).values
    y = df['p_avg'].values

    return X, y

# Prepare the datasets
X_bray_global, y_bray_global = prepare_data(bray_global)
X_bray_afrlac, y_bray_afrlac = prepare_data(bray_afrlac)
X_olsen_afrlac, y_olsen_afrlac = prepare_data(olsen_afrlac)
X_olsen_global, y_olsen_global = prepare_data(olsen_global)
X_stp_afrlac, y_stp_afrlac = prepare_data(stp_afrlac)
X_stp_global, y_stp_global = prepare_data(stp_global)


In [111]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_olsen_afrlac, y_bray_afrlac, test_size=0.2, random_state=42)

In [113]:
# Define the input shape
input_shape = X_train.shape[1]

# Define the autoencoder architecture
input_data = Input(shape=(input_shape,))
encoded = Dense(64, activation='relu')(input_data)
encoded = Dense(32, activation='relu')(encoded)
decoded = Dense(64, activation='relu')(encoded)
decoded = Dense(input_shape, activation='sigmoid')(decoded)

# Create the autoencoder model
autoencoder = Model(inputs=input_data, outputs=decoded)

# Compile the model
autoencoder.compile(optimizer='adam', loss='mse')

# Fit the model
autoencoder.fit(X_train, X_train, epochs=100, batch_size=32, validation_split=0.2)

# Get the encoded features
encoder = Model(inputs=input_data, outputs=encoded)
encoded_features = encoder.predict(X_train)


Epoch 1/100
5/5 [==============================] - 1s 23ms/step - loss: 196471280.0000 - val_loss: 230966480.0000
Epoch 2/100
5/5 [==============================] - 0s 6ms/step - loss: 196471200.0000 - val_loss: 230966464.0000
Epoch 3/100
5/5 [==============================] - 0s 8ms/step - loss: 196471184.0000 - val_loss: 230966464.0000
Epoch 4/100
5/5 [==============================] - 0s 7ms/step - loss: 196471184.0000 - val_loss: 230966464.0000
Epoch 5/100
5/5 [==============================] - 0s 10ms/step - loss: 196471184.0000 - val_loss: 230966464.0000
Epoch 6/100
5/5 [==============================] - 0s 8ms/step - loss: 196471184.0000 - val_loss: 230966464.0000
Epoch 7/100
5/5 [==============================] - 0s 8ms/step - loss: 196471184.0000 - val_loss: 230966464.0000
Epoch 8/100
5/5 [==============================] - 0s 8ms/step - loss: 196471184.0000 - val_loss: 230966464.0000
Epoch 9/100
5/5 [==============================] - 0s 7ms/step - loss: 196471168.0000 - val_lo

Epoch 73/100
5/5 [==============================] - 0s 6ms/step - loss: 196471184.0000 - val_loss: 230966432.0000
Epoch 74/100
5/5 [==============================] - 0s 8ms/step - loss: 196471168.0000 - val_loss: 230966432.0000
Epoch 75/100
5/5 [==============================] - 0s 7ms/step - loss: 196471184.0000 - val_loss: 230966432.0000
Epoch 76/100
5/5 [==============================] - 0s 7ms/step - loss: 196471184.0000 - val_loss: 230966432.0000
Epoch 77/100
5/5 [==============================] - 0s 6ms/step - loss: 196471184.0000 - val_loss: 230966432.0000
Epoch 78/100
5/5 [==============================] - 0s 6ms/step - loss: 196471184.0000 - val_loss: 230966432.0000
Epoch 79/100
5/5 [==============================] - 0s 7ms/step - loss: 196471168.0000 - val_loss: 230966432.0000
Epoch 80/100
5/5 [==============================] - 0s 6ms/step - loss: 196471184.0000 - val_loss: 230966432.0000
Epoch 81/100
5/5 [==============================] - 0s 7ms/step - loss: 196471168.0000 -

In [114]:
# Calculate and print the correlation coefficients
corr_train = np.corrcoef(X_train.flatten(), autoencoder.predict(X_train).flatten())[0, 1]
corr_test = np.corrcoef(X_test.flatten(), autoencoder.predict(X_test).flatten())[0, 1]
print("Correlation coefficient for training data: ", corr_train)
print("Correlation coefficient for test data: ", corr_test)

2/2 [==============================] - 0s 1ms/step
Correlation coefficient for training data:  0.068426439704517
Correlation coefficient for test data:  0.047340566386624004
